In [3]:
!pip install haversine

In [13]:
import pandas as pd
import requests
from haversine import haversine, Unit

In [21]:
class Route:
    def __init__(self, headers={"X-NCP-APIGW-API-KEY-ID": "r97kqovind",
           "X-NCP-APIGW-API-KEY": "FWeecTIqiwQGs2RGil1IPA8wg9QURxjYBzyEPxR7"}):
        self.headers = headers
    
    # 각 출발지 -> 위경도 변환
    def addr_to_xy(self):
        # 주소값 입력
        self.d1_name = input("출발지 1을 입력하세요. : ")
        self.d2_name = input("출발지 2를 입력하세요. : ")
        # URL 설정
        self.d1_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d1_name)
        self.d2_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={}".format(self.d2_name)
        # 리퀘스트
        self.response1 = requests.get(self.d1_url, headers=self.headers)
        self.response2 = requests.get(self.d2_url, headers=self.headers)
        # JSON 파싱하여 위경도 추출
        self.d1_xy = self.response1.json()["addresses"][0]["x"] +"," + self.response1.json()["addresses"][0]["y"]
        self.d2_xy = self.response2.json()["addresses"][0]["x"] +"," + self.response2.json()["addresses"][0]["y"]
        return self.d1_xy, self.d2_xy
    
    # 출발지간 위경도 -> 경로값 변환
    def road_path(self):
        self.d_option = "traoptimal"
        self.d_url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={}&goal={}&option={}".format(self.d1_xy, self.d2_xy, self.d_option)
        self.response = requests.get(self.d_url, headers=self.headers)
        self.path = self.response.json()["route"]["traoptimal"][0]["path"]
        self.avg_path = self.response.json()["route"]["traoptimal"][0]["summary"]["distance"] / len(self.path)
        print("거리 : {}m, ".format(self.response.json()["route"]["traoptimal"][0]["summary"]["distance"]), len(self.path), ", ", self.avg_path)
        return self.path

In [15]:
route = Route()

In [7]:
# 서울시 강남구 도산대로 110
# 서울시 성동구 고산자로 270
# 서울시 성동구 성수이로 113
# 서울시 성동구 아차산로 18
# 경기도 광명시 철산로 57
# 서울 금천구 시흥대로73길 70
# 서울 노원구 노해로 437

# 성수역 -> 뚝섬역     : 거리 :   1453m,   49 ,  29.653061224489797
# 신사역 -> 성수역     : 거리 :   6809m,  193 ,  35.27979274611399
# 신사역 -> 금천구청   : 거리 :  21586m,  462 ,  46.722943722943725
# 금천구청 -> 노원구청 : 거리 :  42402m,  918 ,  46.189542483660134
# 포인트당 평균 약 40m, 중앙을 기점으로 10포인트당 반경 , 거리의 1/3

In [16]:
route.addr_to_xy()

출발지 1을 입력하세요. : 서울시 강남구 도산대로 110
출발지 2를 입력하세요. : 서울시 성동구 고산자로 270


('127.0210472,37.5162749', '127.0368210,37.5634560')

In [17]:
route.road_path()

거리 : 6422m,  186 ,  34.526881720430104


[[127.0208825, 37.5162135],
 [127.020799, 37.5163583],
 [127.020759, 37.5164149],
 [127.0206939, 37.5165156],
 [127.0208735, 37.5165587],
 [127.0212685, 37.5167315],
 [127.0219308, 37.5170317],
 [127.0223473, 37.5172037],
 [127.022713, 37.5173521],
 [127.0228563, 37.5174149],
 [127.0229104, 37.5174394],
 [127.023126, 37.5175331],
 [127.0234216, 37.5176623],
 [127.0234939, 37.5176933],
 [127.0242353, 37.5180063],
 [127.0246677, 37.5181775],
 [127.0253493, 37.5184741],
 [127.0256628, 37.518634],
 [127.0259491, 37.5188262],
 [127.0261229, 37.5188909],
 [127.0261387, 37.5188982],
 [127.0263069, 37.5189602],
 [127.026316, 37.5189638],
 [127.0270124, 37.5192433],
 [127.027193, 37.5193152],
 [127.0274233, 37.5194117],
 [127.0276513, 37.5195055],
 [127.0279414, 37.5196238],
 [127.0280305, 37.5196575],
 [127.0280746, 37.5196748],
 [127.0283287, 37.5197497],
 [127.028986, 37.5199534],
 [127.0290369, 37.5199662],
 [127.0300723, 37.5203372],
 [127.0303354, 37.520431],
 [127.0304946, 37.5204776],
 

In [18]:
# 중간 경로의 모든 값
middle_path = route.path[len(route.path)//3:int(len(route.path)//(3/2))]

# 중간 경로 중 크롤링 포인트
crawling_point = middle_path[::10]
crawling_point

[[127.0333992, 37.5263579],
 [127.0337783, 37.5290914],
 [127.0346196, 37.5348543],
 [127.0361719, 37.5438983],
 [127.0365972, 37.5470655],
 [127.0352725, 37.5514743],
 [127.0343784, 37.5547138]]

In [12]:
mid_lng_min = min([data[0] for data in middle_path])
mid_lng_max = max([data[0] for data in middle_path])
mid_lat_min = min([data[1] for data in middle_path])
mid_lat_max = max([data[1] for data in middle_path])
mid_lng_min, mid_lng_max, mid_lat_min, mid_lat_max

(127.0333992, 127.0366307, 37.5263579, 37.5554993)